In [406]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [407]:
df_week = pd.read_csv("SAP_weekly_return_volatility.csv")
df_week.head(5)

,Year,Week_Number,mean_return,volatility,label
0,2019,0,-0.1050,3.594650,0
1,2019,1,0.6532,1.153409,1
2,2019,2,0.7326,1.198802,1
3,2019,3,0.0735,1.593999,0
4,2019,4,-0.3746,2.056743,0


In [408]:
df_daily = pd.read_csv("SAP_weekly_return_volatility_detailed.csv")
df_daily.head(5)

,High,Low,Open,Close,Volume,Adj Close,Return,Date,Week_Number,Year,Day,Weekday,mean_return,volatility
0,99.559998,98.339996,98.570000,99.180000,506300.0,94.896118,0.000,2019-01-02,0,2019,2,Wednesday,-0.1050,3.594650
1,96.830002,95.449997,96.730003,95.459999,807800.0,91.336800,-3.751,2019-01-03,0,2019,3,Thursday,-0.1050,3.594650
2,99.199997,96.910004,97.339996,98.739998,1038100.0,94.475128,3.436,2019-01-04,0,2019,4,Friday,-0.1050,3.594650
3,100.190002,98.760002,99.440002,99.709999,861600.0,95.403229,0.982,2019-01-07,1,2019,7,Monday,0.6532,1.153409
4,101.480003,100.430000,101.370003,101.269997,548600.0,96.895844,1.565,2019-01-08,1,2019,8,Tuesday,0.6532,1.153409


In [409]:
df_week_2019 = df_week[df_week["Year"] == 2019].reset_index(drop = True)
df_week_2020 = df_week[df_week["Year"] == 2020].reset_index(drop = True)

In [410]:
df_week_copy = df_week.copy()
week_ret = list()
for i in range(len(df_week)):
    y = df_week["Year"][i]
    w = df_week["Week_Number"][i]
    week_ret.append(np.array(df_daily["Adj Close"][(df_daily["Year"] == y) & (df_daily["Week_Number"] == w)])[-1])
    
df_week_copy["week_fri_ret"] = week_ret
df_week_copy.head(5)

,Year,Week_Number,mean_return,volatility,label,week_fri_ret
0,2019,0,-0.1050,3.594650,0,94.475128
1,2019,1,0.6532,1.153409,1,97.575188
2,2019,2,0.7326,1.198802,1,101.172775
3,2019,3,0.0735,1.593999,0,101.431114
4,2019,4,-0.3746,2.056743,0,99.460091


## Q 1. compute the contributions of μ and σ for logistic regression, Euclidean kNN and (degree 1) linear model. Summarize them in a table and discuss your findings.

### KNN

In [411]:
def knn_label(x_train_knn,y_train_knn,x_test_knn,y_test_knn, k):  
    x = x_train_knn.values
    y = y_train_knn.values
    
    scaler = StandardScaler().fit(x)
    x = scaler.transform(x)
    
    neigh = KNeighborsClassifier(n_neighbors = k)
    neigh.fit(x,y)
    
    new_scaled_x_test = scaler.transform(x_test_knn)
    y_predict = neigh.predict(new_scaled_x_test)
    acc = round(np.mean(y_test_knn.values == y_predict) * 100,2)
    
    return [acc]

In [412]:
x_knn_train_mv = df_week_2019[["mean_return","volatility"]]
y_knn_train_mv = df_week_2019["label"]

x_knn_test_mv = df_week_2020[["mean_return","volatility"]]

y_knn_test_mv = df_week_2020["label"]

knn_ret_mv_acc = knn_label(x_knn_train_mv, y_knn_train_mv, x_knn_test_mv ,y_knn_test_mv, 3)
knn_ret_mv_acc[0]

86.79

In [413]:
x_knn_train_m = df_week_2019[["mean_return"]]
y_knn_train_m = df_week_2019["label"]

x_knn_test_m = df_week_2020[["mean_return"]]

y_knn_test_m = df_week_2020["label"]

knn_ret_m_acc = knn_label(x_knn_train_m, y_knn_train_m, x_knn_test_m ,y_knn_test_m, 3)
knn_ret_m_acc[0]

84.91

In [414]:
x_knn_train_v = df_week_2019[["volatility"]]
y_knn_train_v = df_week_2019["label"]

x_knn_test_v = df_week_2020[["volatility"]]

y_knn_test_v = df_week_2020["label"]

knn_ret_v_acc = knn_label(x_knn_train_v, y_knn_train_v, x_knn_test_v ,y_knn_test_v, 3)
knn_ret_v_acc[0]

66.04

### Logistic Regression

In [415]:
def lr_prediction(x_train_lr, y_train_lr, x_test_lr, y_test_lr):   
    x_lr = x_train_lr.values
    y_lr = y_train_lr.values
    
    scaler_lr = StandardScaler().fit(x_lr)
    x_lr = scaler_lr.transform(x_lr)
    
    log_reg_clss = LogisticRegression()
    log_reg_clss.fit(x_lr,y_lr)
    
    new_scaled_x_lr = scaler_lr.transform(np.asmatrix(x_test_lr))
    p_lr = log_reg_clss.predict(new_scaled_x_lr)
    acc_lr = round(log_reg_clss.score(x_lr, y_lr) * 100,2)
    
    return [y_test_lr.values,p_lr,acc_lr] 

In [416]:
x_lr_train_mv = df_week_2019[["mean_return","volatility"]]
y_lr_train_mv = df_week_2019["label"]

x_lr_test_mv = df_week_2020[["mean_return","volatility"]]

y_lr_test_mv = df_week_2020["label"]

lr_mv_acc = lr_prediction(x_lr_train_mv, y_lr_train_mv, x_lr_test_mv ,y_lr_test_mv)[-1]
lr_mv_acc

86.79

In [417]:
x_lr_train_m = df_week_2019[["mean_return"]]
y_lr_train_m = df_week_2019["label"]

x_lr_test_m = df_week_2020[["mean_return"]]

y_lr_test_m = df_week_2020["label"]

lr_m_acc = lr_prediction(x_lr_train_m, y_lr_train_m, x_lr_test_m ,y_lr_test_m)[-1]
lr_m_acc

71.7

In [418]:
x_lr_train_v = df_week_2019[["volatility"]]
y_lr_train_v = df_week_2019["label"]

x_lr_test_v = df_week_2020[["volatility"]]

y_lr_test_v = df_week_2020["label"]

lr_v_acc = lr_prediction(x_lr_train_v, y_lr_train_v, x_lr_test_v ,y_lr_test_v)[-1]
lr_v_acc

64.15

In [419]:
knn_stock_list = [knn_ret_mv_acc[0] - knn_ret_m_acc[0],knn_ret_mv_acc[0] - knn_ret_v_acc[0]]
lr_stock_list = [lr_mv_acc - lr_m_acc,lr_mv_acc - lr_v_acc]

df_contri_stock = pd.DataFrame([knn_stock_list,lr_stock_list], index = ["KNN","LR"], \
                               columns = ["Without mean","Without Volatility"]).T
df_contri_stock

,KNN,LR
Without mean,1.88,15.09
Without Volatility,20.75,22.64


* In cases of both KNN classifier and Logistic Regression volatility has the bigger contribution in overall accuracy.

## Q 2. take IRIS dataset.

In [420]:
df_iris = pd.read_csv("Iris.csv")
df_iris.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [421]:
df_setosa = df_iris.copy().reset_index(drop = True)
df_setosa["label"] = np.where(df_setosa["Species"] == "Iris-setosa",1,0)

df_virginica = df_iris.copy().reset_index(drop = True)
df_virginica["label"] = np.where(df_setosa["Species"] == "Iris-virginica",1,0)

df_versicolor = df_iris.copy().reset_index(drop = True)
df_versicolor["label"] = np.where(df_setosa["Species"] == "Iris-versicolor",1,0)

### Logistic Regression for IRIS

In [422]:
def data_split(tts):
    x_train, x_test, y_train, y_test = tts
    return [x_train, y_train, x_test, y_test]

In [423]:
setosa = data_split(train_test_split(df_setosa[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_setosa["label"], test_size = 0.5,random_state=0))
setosa_acc = lr_prediction(setosa[0],setosa[1],setosa[2],setosa[3])
setosa_acc[2]

100.0

#### Setosa's calculation

In [424]:
setosa_sl = data_split(train_test_split(df_setosa[["SepalWidthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_setosa["label"], test_size = 0.5,random_state=0))
setosa_sl_acc = lr_prediction(setosa_sl[0],setosa_sl[1],setosa_sl[2],setosa_sl[3])

setosa_sw = data_split(train_test_split(df_setosa[["SepalLengthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_setosa["label"], test_size = 0.5,random_state=0))
setosa_sw_acc = lr_prediction(setosa_sw[0],setosa_sw[1],setosa_sw[2],setosa_sw[3])

setosa_pl = data_split(train_test_split(df_setosa[["SepalLengthCm","SepalWidthCm","PetalWidthCm"]],\
                                     df_setosa["label"], test_size = 0.5,random_state=0))
setosa_pl_acc = lr_prediction(setosa_pl[0],setosa_pl[1],setosa_pl[2],setosa_pl[3])

setosa_pw = data_split(train_test_split(df_setosa[["SepalLengthCm","SepalWidthCm","PetalLengthCm"]],\
                                     df_setosa["label"], test_size = 0.5,random_state=0))
setosa_pw_acc = lr_prediction(setosa_pw[0],setosa_pw[1],setosa_pw[2],setosa_pw[3])

In [425]:
s_sl = setosa_acc[2] - setosa_sl_acc[2]
s_sw = setosa_acc[2] - setosa_sw_acc[2]
s_pl = setosa_acc[2] - setosa_pl_acc[2]
s_pw = setosa_acc[2] - setosa_pw_acc[2]
s_list = [s_sl,s_sw,s_pl,s_pw]

s_list_per = [setosa_acc[2],setosa_sl_acc[2],setosa_sw_acc[2],setosa_pl_acc[2],setosa_pw_acc[2]]

#### Virginica calculations

In [426]:
virginica = data_split(train_test_split(df_virginica[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_virginica["label"], test_size = 0.5,random_state=0))
virginica_acc = lr_prediction(virginica[0],virginica[1],virginica[2],virginica[3])

virginica_sl = data_split(train_test_split(df_virginica[["SepalWidthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_virginica["label"], test_size = 0.5,random_state=0))
virginica_sl_acc = lr_prediction(virginica_sl[0],virginica_sl[1],virginica_sl[2],virginica_sl[3])

virginica_sw = data_split(train_test_split(df_virginica[["SepalLengthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_virginica["label"], test_size = 0.5,random_state=0))
virginica_sw_acc = lr_prediction(virginica_sw[0],virginica_sw[1],virginica_sw[2],virginica_sw[3])

virginica_pl = data_split(train_test_split(df_virginica[["SepalLengthCm","SepalWidthCm","PetalWidthCm"]],\
                                     df_virginica["label"], test_size = 0.5,random_state=0))
virginica_pl_acc = lr_prediction(virginica_pl[0],virginica_pl[1],virginica_pl[2],virginica_pl[3])

virginica_pw = data_split(train_test_split(df_virginica[["SepalLengthCm","SepalWidthCm","PetalLengthCm"]],\
                                     df_virginica["label"], test_size = 0.5,random_state=0))
virginica_pw_acc = lr_prediction(virginica_pw[0],virginica_pw[1],virginica_pw[2],virginica_pw[3])


In [427]:
vi_sl = virginica_acc[2] - virginica_sl_acc[2]
vi_sw = virginica_acc[2] - virginica_sw_acc[2]
vi_pl = virginica_acc[2] - virginica_pl_acc[2]
vi_pw = virginica_acc[2] - virginica_pw_acc[2]
v_list = [vi_sl,vi_sw,vi_pl,vi_pw]

v_list_per = [virginica_acc[2],virginica_sl_acc[2],virginica_sw_acc[2],virginica_pl_acc[2],virginica_pw_acc[2]]

### Versicolor calculations

In [428]:
versicolor = data_split(train_test_split(df_versicolor[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_versicolor["label"], test_size = 0.5,random_state=0))
versicolor_acc = lr_prediction(versicolor[0],versicolor[1],versicolor[2],versicolor[3])

versicolor_sl = data_split(train_test_split(df_versicolor[["SepalWidthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_versicolor["label"], test_size = 0.5,random_state=0))
versicolor_sl_acc = lr_prediction(versicolor_sl[0],versicolor_sl[1],versicolor_sl[2],versicolor_sl[3])

versicolor_sw = data_split(train_test_split(df_versicolor[["SepalLengthCm","PetalLengthCm","PetalWidthCm"]],\
                                     df_versicolor["label"], test_size = 0.5,random_state=0))
versicolor_sw_acc = lr_prediction(versicolor_sw[0],versicolor_sw[1],versicolor_sw[2],versicolor_sw[3])

versicolor_pl = data_split(train_test_split(df_versicolor[["SepalLengthCm","SepalWidthCm","PetalWidthCm"]],\
                                     df_versicolor["label"], test_size = 0.5,random_state=0))
versicolor_pl_acc = lr_prediction(versicolor_pl[0],versicolor_pl[1],versicolor_pl[2],versicolor_pl[3])

versicolor_pw = data_split(train_test_split(df_versicolor[["SepalLengthCm","SepalWidthCm","PetalLengthCm"]],\
                                     df_versicolor["label"], test_size = 0.5,random_state=0))
versicolor_pw_acc = lr_prediction(versicolor_pw[0],versicolor_pw[1],versicolor_pw[2],versicolor_pw[3])


In [429]:
ve_sl = versicolor_acc[2] - versicolor_sl_acc[2]
ve_sw = versicolor_acc[2] - versicolor_sw_acc[2]
ve_pl = versicolor_acc[2] - versicolor_pl_acc[2]
ve_pw = versicolor_acc[2] - versicolor_pw_acc[2]
ve_list = [ve_sl,ve_sw,ve_pl,ve_pw]

ve_list_per = [versicolor_acc[2],versicolor_sl_acc[2],versicolor_sw_acc[2],versicolor_pl_acc[2],versicolor_pw_acc[2]]

In [430]:
flower = ["Sepal Length","Sepal Width","Petal Length","Petal Width"]
marginal = pd.DataFrame([flower,s_list,v_list,ve_list], index = ["Flower","Setosa","Virginica","Versicolor"],).T

In [431]:
features = ["All Features","Sepal Length","Sepal Width","Petal Length","Petal Width"]
percentage = pd.DataFrame([features,s_list_per,v_list_per,ve_list_per], index = ["Flower","Setosa","Virginica","Versicolor"],).T
percentage

,Flower,Setosa,Virginica,Versicolor
0,All Features,100.0,97.33,82.67
1,Sepal Length,100.0,97.33,82.67
2,Sepal Width,100.0,97.33,73.33
3,Petal Length,100.0,97.33,82.67
4,Petal Width,100.0,93.33,81.33


In [432]:
marginal

,Flower,Setosa,Virginica,Versicolor
0,Sepal Length,0.0,0.0,0.0
1,Sepal Width,0.0,0.0,9.34
2,Petal Length,0.0,0.0,0.0
3,Petal Width,0.0,4.0,1.34
